# Crawling UAS

## Import Library

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


*   **requests** : Ini adalah sebuah modul dalam Python yang digunakan untuk membuat permintaan HTTP, seperti mengambil konten dari URL atau berkomunikasi dengan API.
*   **BeautifulSoup** : Pustaka ini digunakan untuk melakukan web scraping atau ekstraksi data dari dokumen HTML. Ini membantu dalam mengekstrak informasi dari halaman web dengan lebih mudah, memanfaatkan struktur HTML.
*   **pandas** : Pustaka ini sangat berguna dalam analisis data. Ini menyediakan struktur data yang mudah digunakan dan banyak alat untuk membersihkan, mengubah, dan menganalisis data, terutama dalam bentuk tabel dan dataframe.





In [17]:
def get_data(url, category):
  data_page = requests.get(url)
  print(url)
  if data_page.status_code == 200:
    data_berita = []
    container = BeautifulSoup(data_page.content, 'html.parser')
    container_page = container.find('div', class_='detail__body-text itp_bodycontent')
    data = container_page.find_all('p')
    judul_berita = container.find('h1', class_='detail__title').text.strip()
    tanggal_berita = container.find('div', class_='detail__date').text.strip()

    data_content = ''.strip()
    for i in data:
      if  'ADVERTISEMENT' not in i.text.strip() and  'SCROLL TO RESUME CONTENT' not in i.text.strip() and i.find('div', class_='detail__multiple') == None:
        data_content += i.text.strip() + ' '

    data_berita.append(judul_berita)
    data_berita.append(tanggal_berita)
    data_berita.append(data_content)
    data_berita.append(category)

    return data_berita

Fungsi `get_data` untuk mengambil data dari sebuah URL yang diberikan dan mengekstrak informasi tertentu dari halaman web tersebut.

1. **Import Libraries**: `requests` dan `BeautifulSoup` untuk melakukan permintaan HTTP dan untuk melakukan *web scraping* (mengekstrak data dari halaman web), secara berurutan.

2. **HTTP Request**: Fungsi dimulai dengan mengirimkan permintaan HTTP ke URL yang diberikan dengan menggunakan `requests.get(url)`. Respons dari permintaan ini diambil untuk dianalisis.

3. **Pemeriksaan Kode Status**: Kode status dari permintaan HTTP diperiksa dengan `data_page.status_code`. Jika status kode adalah 200 (yang berarti permintaan berhasil), maka proses ekstraksi data dilanjutkan. Kode status 200 menandakan bahwa halaman web telah berhasil dimuat.

4. **Ekstraksi Data**: Menggunakan `BeautifulSoup`, konten dari halaman web diuraikan untuk menemukan elemen tertentu:
   - **Judul Berita**: Diambil dari elemen dengan kelas CSS `detail__title`.
   - **Tanggal Berita**: Diambil dari elemen dengan kelas CSS `detail__date`.
   - **Konten Berita**: Dalam loop `for` yang mengiterasi melalui elemen-elemen `<p>`, konten berita diambil. Namun, beberapa kondisi diperiksa, seperti memastikan teks tidak mengandung kata 'ADVERTISEMENT' atau 'SCROLL TO RESUME CONTENT', dan mengecualikan elemen dengan kelas `detail__multiple`.

5. **Penyimpanan Data**: Semua informasi yang telah diekstrak (judul, tanggal, konten, dan kategori) disimpan dalam list `data_berita`.

6. **Pengembalian Data**: Setelah informasi dikumpulkan dalam list `data_berita`, data tersebut dikembalikan menggunakan pernyataan `return`.

Fungsi ini dirancang untuk mengambil berita dari suatu URL, membersihkan teks dari konten yang tidak diinginkan atau iklan, dan mengembalikan informasi berita beserta kategorinya.

In [18]:
def get_content (url, category):
  menu_page = requests.get(url)
  if menu_page.status_code == 200:
    content = BeautifulSoup(menu_page.content, 'html.parser')
    content_page = content.find('div', class_='grid-row list-content')
    container_content = content_page.find_all('div', class_='media media--left media--image-radius block-link')

    # Untuk menyimpan url content
    data_content = []
    for c in container_content:
      link_content = c.find('a', class_='media__link')
      subtitle = c.find('h2', class_='media__subtitle')
      if subtitle == None and '20.detik' not in str(link_content) and 'intermeso' not in str(link_content) and 'foto' not in str(link_content):
        data_berita = get_data(link_content.get('href'), category)
        data_content.append(data_berita)

    next_page = content.find_all('a', class_='pagination__item')
    for i in next_page:
      if 'Next' == i.text.strip():
        return (data_content, i.get('href'))

Fungsi `get_content` untuk mengambil konten dari halaman web yang berisi daftar konten dengan kategori tertentu, dan kemudian mengekstrak konten berita dari setiap link yang valid.

1. **HTTP Request**: kode melakukan permintaan HTTP ke URL yang diberikan menggunakan `requests.get(url)` untuk mengunduh halaman web.

2. **Pemeriksaan Kode Status**: Kode status halaman web diperiksa menggunakan `menu_page.status_code`. Jika status kode adalah 200 (berarti permintaan berhasil), proses ekstraksi konten dilanjutkan.

3. **Ekstraksi Konten**: Menggunakan `BeautifulSoup`, konten dari halaman web diuraikan untuk menemukan elemen tertentu:
   - `content_page`: Ini adalah bagian dari halaman yang berisi daftar konten.
   - `container_content`: Di sini, semua konten yang relevan disimpan dalam `container_content`.

4. **Loop untuk Konten**: dilakukan loop pada `container_content` untuk setiap elemen yang ditemukan di dalamnya. Setiap elemen tersebut diuji untuk memastikan bahwa linknya valid dan bukan merupakan jenis konten tertentu (seperti 'foto' atau halaman '20.detik') yang tidak diinginkan.

5. **Ekstraksi Data Berita**: Jika link terbukti valid, fungsi `get_data` dipanggil untuk mengambil informasi terkait berita dari link tersebut menggunakan fungsi yang telah kita bahas sebelumnya (`get_data(link_content.get('href'), category)`). Informasi ini ditambahkan ke dalam list `data_content`.

6. **Mencari Halaman Berikutnya**: Setelah semua konten dari halaman ini diolah, kode mencari tautan untuk halaman berikutnya jika ada. Jika ditemukan tautan 'Next', maka fungsi mengembalikan tuple yang berisi list `data_content` yang berisi konten yang sudah diambil, serta URL dari halaman berikutnya.

Fungsi ini mengambil konten dari halaman web yang berisi daftar konten, mengekstrak berita dari setiap link yang valid, dan mengembalikan data berita beserta URL halaman berikutnya untuk dilanjutkan ke proses ekstraksi data berikutnya.

In [25]:
def scraping_detik_news(url='https://news.detik.com/indeks', num_content = 50, category= ['Edu']):
  response = requests.get(url)

  # Untuk mengecek respon url dan mencari menu sidebar
  if response.status_code == 200:
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    menu = soup.find('nav', class_='static-nav')
    link_menu = menu.find_all('a')
    # Untuk menyimpan url navbar
    save_link_navbar = []
    for link in link_menu:
      if 'travel' not in link.get('href') and 'wolipop' not in link.get('href'):
        save_link_navbar.append([link.find('div').text.strip(), link.get('href')])

    # Untuk mengecek respon url dan mencari link content
    data_semua_berita = []
    for nav in save_link_navbar:
      if nav[0] in category:
        data_berita, next_page = get_content(nav[1], nav[0])
        data_semua_berita += data_berita


        next_page_more = next_page
        for i in range(num_content-1):
          more_content, next_page_more = get_content(next_page_more, nav[0])
          data_semua_berita += more_content

    df = pd.DataFrame(data_semua_berita, columns=['Judul', 'Tanggal', 'Artikel', 'Category'])
    df.to_csv('Data Berita Detik EDU.csv', index=False)
    print(f'Data Berhasil Di Simpan dengan Jumlah {len(data_semua_berita)}')

In [27]:
def scraping_detik_news(url='https://news.detik.com/indeks', num_content = 50, category= ['Sport']):
  response = requests.get(url)

  # Untuk mengecek respon url dan mencari menu sidebar
  if response.status_code == 200:
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    menu = soup.find('nav', class_='static-nav')
    link_menu = menu.find_all('a')
    # Untuk menyimpan url navbar
    save_link_navbar = []
    for link in link_menu:
      if 'travel' not in link.get('href') and 'wolipop' not in link.get('href'):
        save_link_navbar.append([link.find('div').text.strip(), link.get('href')])

    # Untuk mengecek respon url dan mencari link content
    data_semua_berita = []
    for nav in save_link_navbar:
      if nav[0] in category:
        data_berita, next_page = get_content(nav[1], nav[0])
        data_semua_berita += data_berita


        next_page_more = next_page
        for i in range(num_content-1):
          more_content, next_page_more = get_content(next_page_more, nav[0])
          data_semua_berita += more_content

    df = pd.DataFrame(data_semua_berita, columns=['Judul', 'Tanggal', 'Artikel', 'Category'])
    df.to_csv('Data Berita Detik SPORT.csv', index=False)
    print(f'Data Berhasil Di Simpan dengan Jumlah {len(data_semua_berita)}')

In [29]:
def scraping_detik_news(url='https://news.detik.com/indeks', num_content = 25, category= ['Food']):
  response = requests.get(url)

  # Untuk mengecek respon url dan mencari menu sidebar
  if response.status_code == 200:
    html = response.content
    soup = BeautifulSoup(html, 'html.parser')
    menu = soup.find('nav', class_='static-nav')
    link_menu = menu.find_all('a')
    # Untuk menyimpan url navbar
    save_link_navbar = []
    for link in link_menu:
      if 'travel' not in link.get('href') and 'wolipop' not in link.get('href'):
        save_link_navbar.append([link.find('div').text.strip(), link.get('href')])

    # Untuk mengecek respon url dan mencari link content
    data_semua_berita = []
    for nav in save_link_navbar:
      if nav[0] in category:
        data_berita, next_page = get_content(nav[1], nav[0])
        data_semua_berita += data_berita


        next_page_more = next_page
        for i in range(num_content-1):
          more_content, next_page_more = get_content(next_page_more, nav[0])
          data_semua_berita += more_content

    df = pd.DataFrame(data_semua_berita, columns=['Judul', 'Tanggal', 'Artikel', 'Category'])
    df.to_csv('Data Berita Detik FOOD.csv', index=False)
    print(f'Data Berhasil Di Simpan dengan Jumlah {len(data_semua_berita)}')

Fungsi `scraping_detik_news` untuk melakukan *web scraping* pada situs web Detik News, mengambil berita dari berbagai kategori tertentu, kemudian menyimpan informasi tersebut dalam format CSV. Ini dilakukan dalam beberapa langkah :

1. **Mengambil Menu Sidebar**: Pertama, kode mengambil menu sidebar dari halaman utama menggunakan BeautifulSoup. Ini memungkinkan untuk mengekstrak daftar kategori dan tautan menu yang terkait.

2. **Mengecek Kategori**: Setelah tautan-tautan menu ditemukan, kode memeriksa setiap tautan dan memeriksa apakah kategori berita pada tautan tersebut sesuai dengan kategori yang diberikan pada parameter fungsi.

3. **Ekstraksi Konten Berita**: Jika kategori berita sesuai dengan yang diminta, fungsi `get_content` dipanggil untuk mengekstrak konten berita dari setiap tautan tersebut. Konten yang diambil akan ditambahkan ke dalam list `data_semua_berita`.

4. **Iterasi untuk Lebih Banyak Konten**: Kemudian, kode mengambil lebih banyak konten berita dari halaman berikutnya (jika ada) menggunakan fungsi `get_content`, dengan memanfaatkan tautan "Next" dari halaman tersebut. Ini dilakukan dalam loop, mengambil konten tambahan sampai jumlah yang diminta (`num_content`) terpenuhi.

5. **Pembuatan DataFrame dan Penyimpanan Data**: Setelah semua konten berita dikumpulkan, data diorganisir dalam bentuk *dataframe* menggunakan `pandas`. Selanjutnya, data ini disimpan dalam format CSV.

6. **Pesan Konfirmasi**: Terakhir, fungsi memberikan pesan konfirmasi yang menyatakan bahwa data telah berhasil disimpan bersama dengan jumlah total berita yang terkumpul.

Secara keseluruhan, fungsi ini untuk mengambil berita dari situs Detik News, khususnya dari kategori yang telah ditentukan, kemudian menyimpannya dalam format CSV untuk penggunaan atau analisis lebih lanjut.

In [26]:
scraping_detik_news(url='https://news.detik.com/indeks', num_content=50, category=['Edu'])

https://www.detik.com/edu/seleksi-masuk-pt/d-7087628/23-universitas-terbaik-jurusan-kimia-versi-scimago-2023-referensi-snpmb-2024
https://www.detik.com/edu/seleksi-masuk-pt/d-7087601/60-prodi-terketat-di-snbp-dan-snbt-2023-referensi-snpmb-2024
https://www.detik.com/edu/detikpedia/d-7087301/hari-nusantara-13-desember-asal-usul-peringatan-dan-tema-tahun-2023
https://www.detik.com/edu/detikpedia/d-7087425/apa-itu-majelis-umum-pbb-ini-sejarah-dan-tugasnya
https://www.detik.com/edu/perguruan-tinggi/d-7087265/25-ptn-terakreditasi-unggul-ban-pt-bisa-jadi-referensi-snpmb-2024
https://www.detik.com/edu/seleksi-masuk-pt/d-7087377/13-universitas-terbaik-jurusan-psikologi-versi-scimago-2023-buat-persiapan-snpmb
https://www.detik.com/edu/seleksi-masuk-pt/d-7087285/catat-ini-2-kelompok-yang-tidak-boleh-daftar-snbt-2024
https://www.detik.com/edu/detikpedia/d-7086950/mengapa-masyarakat-tolak-pengungsi-rohingya-pakar-ungkap-hoax-yang-beredar
https://www.detik.com/edu/sekolah/d-7087174/idai-indonesia-pe

In [28]:
scraping_detik_news(url='https://news.detik.com/indeks', num_content=50, category=['Sport'])

https://sport.detik.com/raket/d-7087762/fajar-rian-kalahkan-bagas-fikri-kini-alihkan-fokus-ke-partai-kedua
https://sport.detik.com/raket/d-7087760/bwf-world-tour-finals-turnamen-bulutangkis-dengan-hadiah-terbesar
https://sport.detik.com/raket/d-7087563/hasil-bwf-world-tour-finals-2023-apri-fadia-petik-kemenangan
https://sport.detik.com/raket/d-7087335/hasil-bwf-world-tour-finals-fajar-rian-menangi-perang-saudara
https://sport.detik.com/raket/d-7087203/dikalahkan-tai-tzu-ying-gregoria-kurang-lepas-dan-terlalu-hati-hati
https://sport.detik.com/raket/d-7087066/hasil-bwf-world-tour-finals-2023-gregoria-ditaklukkan-tai-tzu-ying
https://sport.detik.com/sport-lain/d-7086619/indonesia-open-aquatic-championship-jadi-ajang-kualifikasi-pon-2024
https://sport.detik.com/sport-lain/d-7086581/citadel-racing-turun-di-balapan-porsche-ini-hasilnya
https://sport.detik.com/sport-lain/d-7086331/menpora-lewat-olahraga-anak-muda-bisa-belajar-jadi-pemimpin
https://sport.detik.com/sport-lain/d-7086642/koi-resp

In [30]:
scraping_detik_news(url='https://news.detik.com/indeks', num_content=25, category=['Food'])

https://food.detik.com/berita-boga/d-7087667/rayakan-natal-dengan-menara-macaroon-hingga-bbq-seru-di-tengah-kota
https://food.detik.com/info-kuliner/d-7087998/12-rekomendasi-all-you-can-eat-paling-favorit-worth-it-di-bogor
https://food.detik.com/info-kuliner/d-7087969/iseng-pria-ini-masak-udang-mentah-di-toilet-pesawat
https://food.detik.com/info-kuliner/d-7087531/ngakak-pria-ini-pernah-rebutan-bihun-goreng-sama-bunga-citra-lestari
https://food.detik.com/info-kuliner/d-7087163/5-makanan-viral-sepanjang-tahun-2023-sudah-pernah-cicip
https://food.detik.com/berita-boga/d-7087119/kisah-jamu-gendong-dari-minuman-kerajaan-hingga-tercatat-di-relief-candi
https://food.detik.com/tempat-makan/d-7087101/begini-lezatnya-tahu-pong-semarang-favorit-orang-terkaya-di-indonesia
https://food.detik.com/info-kuliner/d-7086660/5-pesanan-starbucks-yang-tak-disukai-barista-apa-saja
https://food.detik.com/info-kuliner/d-7085790/gemas-kucing-di-china-ini-hobinya-minta-mie-ke-restoran-setiap-hari
https://food.d

Fungsi `scraping_detik_news` untuk melakukan *web scraping* pada halaman indeks berita Detik dengan parameter sebagai berikut:

- **URL**: `'https://news.detik.com/indeks'`, yang merupakan halaman utama indeks berita Detik.
- **Jumlah Konten**: `num_content=50`, yang merupakan jumlah konten berita yang ingin diambil dari setiap kategori.
- **Kategori**: `['Edu']`, yang menunjukkan bahwa kamu ingin mengambil konten dari kategori 'Edu'.

Fungsi ini akan menjelajahi halaman indeks berita Detik, mengambil konten dari kategori yang diinputkan, dan mencoba mengumpulkan 50 berita dari kategori tersebut. Jika berhasil, hasilnya akan disimpan dalam file CSV.